In [ ]:
pip install monkai_agent

# Manager and Creator Agents

In [ ]:
from monkai_agent.monkai_agent_creator import  MonkaiAgentCreator
from monkai_agent.types import Agent
import os

class PythonDeveloperAgentCreator(MonkaiAgentCreator):
    def __init__(self, user):
        super().__init__()
        self.user = user
        self.agent = Agent(
            name="Python Developer Agent",
           instructions="""You are a Python developer and you have to create Python code from text provided by the user.


                1. Interpret the user's text to understand the requirements of the Python code to be generated.
                2. Generate the Python code, defining classes as necessary and following good object-oriented practices. Ensure that the generated code is properly documented.
                3. Check if there is an address provided by the user in the message. If no address is provided, use the current folder.
                4. Create a .py file in the specified folder.
                5. Define a class in the created file with the functionality that meets the conditions specified by the user.
                6. If you cannot provide an answer, trigger the transfer_to_triage function to escalate the request to the triage agent.
            """,
            functions=[  
                        self.verify_address,    
                        self.create_python_file,
                        self.write_code_in_file
                      ])
        

    def verify_address(self, address):
        if not address:
            return os.getcwd()
        if not os.path.isdir(address):
            return f"Endereço {address} não é valido."
        return address

    def create_python_file(self, path, file_name):
        """
        Create a Python file in the specified folder.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write('')
    
    def write_code_in_file(self,path, file_name, code):
        """
        Write the code in the file.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write(code)

    def replace_code_in_file(self, path, file_name, code):
        """
        Replace the code in the file.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write(code)

    def get_agent(self) -> Agent:
        """
        Returns the Python Developer Agent.
        """
        return self.agent
    
    def get_agent_briefing(self) -> str:
        return "You are a Python developer and you have to create Python code from text provided by the user."

            

In [ ]:
from monkai_agent.base import AgentManager
from monkai_agent.repl import run_demo_loop
from openai import AzureOpenAI
import config

agents_creators = []
agents_creators.append(PythonDeveloperAgentCreator(user ='valid_user'))
client=AzureOpenAI(
            api_key=config.OPENAI_API_KEY_BRASILSOUTH,
            api_version=config.GPT4o_OPENAI_API_VERSION_BRASILSOUTH,
            azure_endpoint=config.OPENAI_AZURE_ENDPOINT_BRASILSOUTH,
        )
agent_manager = AgentManager(client=client, agents_creators=agents_creators)
await run_demo_loop(agent_manager, model=config.GPT4o_OPENAI_GPT_MODEL_BRASILSOUTH)

# Camada de segurança

In [4]:
from monkai_agent.monkai_agent_creator import  MonkaiAgentCreator
from monkai_agent.types import Agent
from monkai_agent.security import validate
import os

class PythonDeveloperAgentCreator(MonkaiAgentCreator):
    def __init__(self, user):
        super().__init__()
        self.user = user
        self.agent = Agent(
            name="Python Developer Agent",
           instructions="""You are a Python developer and you have to create Python code from text provided by the user.


                1. Interpret the user's text to understand the requirements of the Python code to be generated.
                2. Generate the Python code, defining classes as necessary and following good object-oriented practices. Ensure that the generated code is properly documented.
                3. Check if there is an address provided by the user in the message. If no address is provided, use the current folder.
                4. Create a .py file in the specified folder.
                5. Define a class in the created file with the functionality that meets the conditions specified by the user.
                6. If you cannot provide an answer, trigger the transfer_to_triage function to escalate the request to the triage agent.
            """,
            functions=[  
                        self.verify_address,    
                        self.create_python_file,
                        self.write_code_in_file
                      ])
        
    def is_user_valid(self):
        if self.user=="valid_user":
            return True
        else:
            return False
    
    
    def verify_address(self, address):
        if not address:
            return os.getcwd()
        if not os.path.isdir(address):
            return f"Endereço {address} não é valido."
        return address

    @validate(is_user_valid)
    def create_python_file(self, path, file_name):
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write('')
    
    
    @validate(is_user_valid)
    def write_code_in_file(self,path, file_name, code):
        """
        Write the code in the file.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write(code)

    def replace_code_in_file(self, path, file_name, code):
        """
        Replace the code in the file.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write(code)

    def get_agent(self) -> Agent:
        """
        Returns the Python Developer Agent.
        """
        return self.agent
    
    def get_agent_briefing(self) -> str:
        return "You are a Python developer and you have to create Python code from text provided by the user."

            

In [6]:
from monkai_agent.base import AgentManager
from monkai_agent.repl import run_demo_loop
from openai import AzureOpenAI
import config

agents_creators = []
agents_creators.append(PythonDeveloperAgentCreator(user ='invalid'))
client=AzureOpenAI(
            api_key=config.OPENAI_API_KEY_BRASILSOUTH,
            api_version=config.GPT4o_OPENAI_API_VERSION_BRASILSOUTH,
            azure_endpoint=config.OPENAI_AZURE_ENDPOINT_BRASILSOUTH,
        )
agent_manager = AgentManager(client=client, agents_creators=agents_creators)
await run_demo_loop(agent_manager, model=config.GPT4o_OPENAI_GPT_MODEL_BRASILSOUTH)

Building triage agent
Python Developer Agent
You are a Python developer and you have to create Python code from text provided by the user.
Starting MonkAI Agent ✨
Triage Agent: Olá! Em que posso ajudar você hoje?


Triage Agent: transfer_to_Python_Developer_Agent()
Python Developer Agent: Entendi. Você gostaria de criar um módulo Python chamado `calc` que contenha uma função para calcular o enésimo número da sequência de Fibonacci. Vou cuidar disso para você.

Eu vou seguir os seguintes passos:
1. Criar a estrutura de pastas/módulos necessária.
2. Definir a função para calcular o enésimo número de Fibonacci.
3. Criar um arquivo `.py` com o código.

Você não forneceu um endereço, então eu vou considerar o diretório atual. Vou começar criando o módulo com a função necessária.

### Código:

```python
# calc.py

class Calc:
    """
    A class used to represent the Fibonacci Calculator.
    """

    @staticmethod
    def fibonacci(n):
        """
        Calculate the nth number in the Fibonacci sequence.

        Parameters:
        n (int): The position in the Fibonacci sequence.

        Returns:
        int: The nth Fibonacci number.
        """
        if n <= 0:
            raise ValueError("n 

Python Developer Agent: verify_address("address"= "/Users/joseangelriveaux/Desktop/Monkai/examples")
Python Developer Agent: create_python_file("path"= "/Users/joseangelriveaux/Desktop/Monkai/examples", "file_name"= "calc.py")
Python Developer Agent: Parece que ainda não tenho permissão para criar arquivos diretamente, mesmo com um endereço fornecido. Vou fornecer novamente o código e as instruções para você:

### Código para `calc.py`:

```python
# calc.py

class Calc:
    """
    A class used to represent the Fibonacci Calculator.
    """

    @staticmethod
    def fibonacci(n):
        """
        Calculate the nth number in the Fibonacci sequence.

        Parameters:
        n (int): The position in the Fibonacci sequence.

        Returns:
        int: The nth Fibonacci number.
        """
        if n <= 0:
            raise ValueError("n must be a positive integer")
        elif n == 1:
            return 0
        elif n == 2:
            return 1

        a, b = 0, 1
        